<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>

# Deploying a Model for Inference at Production Scale

## 04 - Simple TensorFlow Model
-------

**Table of Contents**

* [Introduction](#introduction)
* [Create Model Directory Structure](#structure)
* [Define a Simple TensorFlow Model](#model)
* [Create Configuration File](#configuration)
* [Load Model in Triton Inference Server](#load)
* [Send Inference Request to Server](#infer)
* [Exercise](#exercise)
* [Conclusion](#conclusion)


<a id="introduction"></a>
### Introduction

In this notebook, we will create a TensorFlow ResNet 50 model, write it out as a `SavedModel` representation and deploy it using Triton Inference Server. We'll see how to create model directory structures and configuration files within Triton Inference Server, how to work with TensorFlow, and how to send inference requests to the models deployed within Triton Inference Server.

<a id="structure"></a>
### Create Model Directory Structure

Triton Inference Server serves models within a model repository. When you first run Triton Inference Server, you'll specify the model repository where the models reside:

```
tritonserver --model-repository=/models
```

Each model resides in its own model subdirectory within the model repository - i.e. each directory within `/models` represents a unique model. For example, in this notebook we'll be deploying a TensorFlow model (`simple-TensorFlow-model`).

All models typically follow a similar directory structure. Within each of these directories, we'll create a configuration file `config.pbtxt` that details information about the model - e.g. batch size, input shapes, deployment backend (PyTorch, ONNX, TensorFlow, TensorRT, etc.) and more. We'll explore the configuration file later in this notebook.

Additionally, we can create one or more versions of our model. Each version lives under a subdirectory names with the respective version number, starting with `1`. It is within this subdirectory where our model files reside (e.g. `model.onnx`, `model.savedmodel`).

```
root@server:/models$ tree
.
├── simple-tensorflow-model
│   ├── 1
│   │   └── model.savedmodel
│   │       ├── assets
│   │       ├── saved_model.pb
│   │       └── variables
│   │           ├── variables.data-00000-of-00001
│   │           └── variables.index

```

We can also add a file representing the names of the outputs. We have omitted this step in this notebook for the sake of brevity. For more details on how to work with model repositories and model directory structures in Triton Inference Server, please see the documentation here: https://github.com/triton-inference-server/server/blob/r20.12/docs/model_repository.md

Below, we'll create the model directory structure for each of our TensorFlow model.



In [ ]:
!mkdir -p models/simple-tensorflow-model/
!mkdir -p models/simple-tensorflow-model/1/

<a id="model"></a>
### Define a Simple TensorFlow Model

In this next section, we'll define a simple TensorFlow ResNet50 model. We'll specify that we will use a pretrained model, which will instantiate the ResNet50 model with the weights learned from training on ImageNet. After defining our `WrappedModel` class, we will instantiate this model, use the private `__call__` method to get the call signature, and use that to save the model using the `tf.saved_model.save` function. 

We'll export our model in an `SavedModel` representation in the version `1` subdirectory of our `simple-tensorflow-model` model directory.

In [ ]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)


class WrappedModel(tf.Module):
    def __init__(self):
        super(WrappedModel, self).__init__()
        self.model = tf.keras.applications.ResNet50()
    @tf.function
    def __call__(self, x):
        return self.model(x)

model = WrappedModel()
call = model.__call__.get_concrete_function(tf.TensorSpec([None, None, None, None], 
                                            tf.float32, name='input_0'))
tf.saved_model.save(model, 
                    'models/simple-tensorflow-model/1/model.savedmodel', 
                    signatures=call)

Next, we'll load the ImageNet labels.

In [ ]:
import json

with open('./imagenet-simple-labels.json') as file:
    labels = json.load(file)

print(labels[:5])

Before working with Triton Inference Server, let's confirm our ResNet50 model pre-trained on ImageNet works on a sample image. We'll use an image of goldfish - feel free to try this with your own images!

In [ ]:
import numpy as np
from PIL import Image


img_path = './assets/goldfish.jpg'
image_pil = Image.open(img_path)
image_pil

Next, we'll import a base ResNet50 model from TensorFlow Keras as well as some helper functions for loading the image, resizing the image, preprocessing the input, and decoding the predictions. We see that our model does exactly as we expect by correctly classifying our image of a goldfish.

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img = image.load_img(img_path, target_size=(224, 224))
image_numpy = image.img_to_array(img)
image_numpy = np.expand_dims(image_numpy, axis=0)
image_numpy = preprocess_input(image_numpy)

preds = model.predict(image_numpy)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

<a id="configuration"></a>
### Create Configuration File

With our model defined and written out in `SavedModel` representation, we now turn our attention to creating configuration files for our models.

A minimal model configuration must specify the name of the model, the platform and/or backend properties, the max_batch_size property, and the input and output tensors of the model (name, data type, and shape).


For more details on how to create model configuration files within Triton Inference Server, please see the documentation: 
https://github.com/triton-inference-server/server/blob/r20.12/docs/model_configuration.md

In [ ]:
configuration = """
name: "simple-tensorflow-model"
platform: "tensorflow_savedmodel"
max_batch_size: 32
input [
 {
    name: "input_0"
    data_type: TYPE_FP32
    format: FORMAT_NHWC
    dims: [ 224, 224, 3 ]
  }
]
output {
    name: "output_0"
    data_type: TYPE_FP32
    dims: [ 1000 ]
  }
"""

with open('models/simple-tensorflow-model/config.pbtxt', 'w') as file:
    file.write(configuration)

<a id="load"></a>
### Load Model in Triton Inference Server


With our model directory structures created, models defined and exported, and configuration files created, we will now wait for Triton Inference Server to load our models. We have set up this lab to use Triton Inference Server in **polling** mode. This means that Triton Inference Server will continuously poll for modifications to our models or for newly created models - once every 30 seconds. Please run the cell below to allow time for Triton Inference Server to poll for new models/modifications before proceeding.

In [ ]:
!sleep 45

At this point, our models should be deployed and ready to use! To confirm Triton Inference Server is up and running, we can see a `curl` request to the below URL.

In [ ]:
!curl -v triton:8000/v2/health/ready

The HTTP request returns status 200 if Triton is ready and non-200 if it is not ready.

We can also send a `curl` request to our model endpoints to confirm our models are deployed and ready to use. This `curl` request returns status 200 if the model is ready and non-200 if it is not ready. 

Additionally, we will also see information about our models such:

* The name of our model,
* The versions available for our model,
* The backend platform (e.g. tensorflow_savedmodel), 
* The inputs and outputs, with their respective names, data types, and shapes.


In [ ]:
!curl -v triton:8000/v2/models/simple-tensorflow-model

<a id="infer"></a>
### Send Inference Request to Server

With our models deployed, it is now time to send inference requests to our models. 

First, we'll load the `tritonclient.http` module and a utility function for working with NumPy data.


In [ ]:
import tritonclient.http as tritonhttpclient
from tritonclient.utils import triton_to_np_dtype

Next, we'll define the input and output names of our model, the name of our model, the URL where our models are deployed with Triton Inference Server (in this case local host of `triton:8000`), and our model version.

In [ ]:
VERBOSE = False
input_name = 'input_0'
input_shape = (1, 224, 224, 3)
input_dtype = 'FP32'
output_name = 'output_0'
model_name = 'simple-tensorflow-model'
url = 'triton:8000'
model_version = '1'

We'll instantiate our client `triton_client` using the `tritonhttpclient.InferenceServerClient` class access the model metadata with the `.get_model_metadata()` method as well as get our model configuration with the `get_model_config()` method.

In [ ]:
triton_client = tritonhttpclient.InferenceServerClient(url=url, verbose=VERBOSE)
model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)
model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)

We'll instantiate a placeholder for our input data using the input name, shape, and data type expected. We'll set the data of the input to be the NumPy array representation of our goldfish image. We'll also instantiate a placeholder for our output data using just the output name.

Lastly, we'll submit our input to the Triton Inference Server using the `triton_client.infer()` method, specifying our model name, model version, inputs, and outputs and convert our result to a NumPy array.

In [ ]:
input0 = tritonhttpclient.InferInput(input_name, input_shape, input_dtype)
input0.set_data_from_numpy(image_numpy, binary_data=False)

output = tritonhttpclient.InferRequestedOutput(output_name, binary_data=False)
response = triton_client.infer(model_name, model_version=model_version, 
                               inputs=[input0], outputs=[output])
logits = response.as_numpy(output_name)
logits = np.asarray(logits, dtype=np.float32)

And that's all there is to it! We can identify the largest logit value and confirm that our model correctly inferred that our image is, indeed, a goldfish.

In [ ]:
print(labels[np.argmax(logits)])

<a id="conclusion"></a>
### Conclusion

In this notebook, we showed how to create a TensorFlow ResNet50 model, write it out as a `SavedModel` representation and deploy it using Triton Inference Server. We saw how to create model directory structures and configuration files within Triton Inference Server, how to work with TensorFlow, and how to send inference requests to the models deployed within Triton Inference Server.

We kindly ask that you do some clean up and run the cell below. This will free up GPU memory for other section of the lab.

In [ ]:
!rm -rf models/simple-tensorflow-model

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./assets/DLI_Header.png"></a></div>